In [1]:
from dotenv import dotenv_values
import os

from agents import setup_sync_agent

env_config = dotenv_values(".local.env")
OPENAI_MODEL_NAME = env_config.get("OPENAI_MODEL_NAME", "text-davinci-003")
OPENAI_API_KEY = env_config["OPENAI_API_KEY"]
DATA_LOCATION = env_config["DATA_LOCATION"]
os.environ["SERP_API_KEY"] = env_config["SERP_API_KEY"]


USERNAME="Div"
USER_EMAIL="div@multion.ai"

In [2]:
chat_history = """\
Harry Potter: Hey all! Very excited to watch Oppenheimer in IMAX this weekend 😍. I’d like to take you guys out to dinner after, my treat. Are you all free?
Hermione Granger: Hmm maybe
Hermione Granger: I wanna watch it really soon though. Friday works for me, not sure about Ron
Harry Potter: If not, let’s do it next week.
Voldemort: Potter, I’m free except Monday and Thursday, if you want an Avada Kedavra for dessert.
Ron Weasley: Yeah next week is cray but you should come to the show Thursday
Ron Weasley: Friday works
Harry Potter:  Let's do Friday. I'm adding Mion to this thread to help schedule dinner."""

chat_history_stage_two = """\
Ron Weasley: I'm allergic to Mandrakes.
Harry Potter: Let's go to Hogsmeade
Voldemort: Sounds good 💀
Hermione Granger: Sounds good"""

chat_history_stage_three = """\
Ron Weasley: Just the vegetarian thing.\
"""

chat_history_stage_four = """\
Ron Weasley: I'm making an executive decision for Italian.\
"""

In [3]:
def run_agent(user, chat_history):
    *chat_history, input_ = chat_history.split("\n")
    chat_history = "\n".join(chat_history)
    #buffer = load_memory_for_user(user)
    # TODO: check if agent is already running for this user, use agent scratchpad if so
    agent_executor = setup_sync_agent(buffer=None, user=user, openai_api_key=OPENAI_API_KEY)
    inputs = {
        "input": input_,
        "chat_history": chat_history,
    }
    resp = agent_executor.run(inputs)
    #new_buffer = agent_executor.memory.buffer
    #save_memory_for_user(user, new_buffer)
    return resp

In [4]:
result1 = run_agent("Harry", chat_history); result1

buffer is  []


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Mion: Hi everyone! I'm Mion and I'm here to help you schedule dinner. It sounds like you'd like to schedule dinner on Friday. Could you please indicate the type of cuisine and the preferred location?

> Finished chain.


"Hi everyone! I'm Mion and I'm here to help you schedule dinner. It sounds like you'd like to schedule dinner on Friday. Could you please indicate the type of cuisine and the preferred location?"

In [5]:
result2 = run_agent("Harry", chat_history + "\nMion: " + result1 + "\n" +  chat_history_stage_two)
result2

buffer is  []


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Take Note
Action Input: Ron Weasley is allergic to Mandrakes.No user foundDo I need to use a tool? No
Mion: Great! We have a consensus on the location, Hogsmeade. Now, could you please indicate the type of cuisine you'd prefer for the dinner? Please remember that Ron is allergic to Mandrakes.

> Finished chain.


"Great! We have a consensus on the location, Hogsmeade. Now, could you please indicate the type of cuisine you'd prefer for the dinner? Please remember that Ron is allergic to Mandrakes."

In [6]:
result3 = run_agent(
    "Harry", 
    chat_history + "\nMion: " + result1 + "\n" +  chat_history_stage_two
    + "\nMion: " + result2 + "\n" + chat_history_stage_three
)
result3

buffer is  []


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Mion: Noted, Ron. So, we're looking for a vegetarian restaurant in Hogsmeade. Does everyone agree with this?

> Finished chain.


"Noted, Ron. So, we're looking for a vegetarian restaurant in Hogsmeade. Does everyone agree with this?"

In [7]:
result4 = run_agent(
    "Harry", 
    chat_history + "\nMion: " + result1 + "\n" +  chat_history_stage_two
    + "\nMion: " + result2 + "\n" + chat_history_stage_three
    + "\nMion: " + str(result3) + "\n" + chat_history_stage_four
)
result4

buffer is  []


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Google Maps Search
Action Input: Italian vegetarian restaurant Hogsmeade[{'title': 'Three Broomsticks', 'description': "Harry Potter-themed eatery at Universal. Popular Harry Potter-themed cafeteria selling British eats in Universal's Islands of Adventure."}, {'title': 'Plant Based Mafia', 'description': None}, {'title': 'Mythos Restaurant', 'description': "Mythical undersea-themed American dining. Overlooking Universal Studios' Inland Sea, this mythic underwater-themed spot serves American fare."}]
[{'title': 'Three Broomsticks', 'description': "Harry Potter-themed eatery at Universal. Popular Harry Potter-themed cafeteria selling British eats in Universal's Islands of Adventure."}, {'title': 'Plant Based Mafia', 'description': None}, {'title': 'Mythos Restaurant', 'description': "Mythical undersea-themed American dining. Overlooking Universal Studios' Inland Sea, this mythic underwater

[{'title': 'Three Broomsticks',
  'description': "Harry Potter-themed eatery at Universal. Popular Harry Potter-themed cafeteria selling British eats in Universal's Islands of Adventure."},
 {'title': 'Plant Based Mafia', 'description': None},
 {'title': 'Mythos Restaurant',
  'description': "Mythical undersea-themed American dining. Overlooking Universal Studios' Inland Sea, this mythic underwater-themed spot serves American fare."}]